## Homework Assignment 1: Analyzing Sales Data

In [16]:
import pandas as pd
import numpy as np

saleinfo = pd.read_csv('sales_data.csv')
saleinfo.head()

,Date,Product,Category,Quantity,Price
0,2023-01-01,Laptop,Electronics,10,800
1,2023-01-01,T-Shirt,Clothing,5,20
2,2023-01-02,Smartphone,Electronics,8,400
3,2023-01-02,Coffee Maker,Home,12,50
4,2023-01-03,Jeans,Clothing,15,30


In [9]:

category_stats = saleinfo.groupby('Category').agg(
    total_quantity=('Quantity','sum'),
    avg_price=('Price','mean'),
    max_quantity=('Quantity','max')
).reset_index()
print(category_stats)

      Category  total_quantity   avg_price  max_quantity
0     Clothing             157   31.176471            15
1  Electronics             183  276.764706            15
2         Home             144   55.000000            14


In [13]:
top_products = saleinfo.groupby(['Category', 'Product'])['Quantity'].sum().reset_index()
top_products = top_products.loc[top_products.groupby('Category')['Quantity'].idxmax()]

print(top_products)

       Category          Product  Quantity
8      Clothing            Jeans        15
27  Electronics         Smart TV        15
46         Home  Pressure Cooker        14


In [14]:
saleinfo['Total_Sales'] = saleinfo['Quantity'] * saleinfo['Price']
date_max_sale = saleinfo.groupby('Date')['Total_Sales'].sum().idxmax()
print(date_max_sale)

2023-01-07


## Homework Assignment 2: Examining Customer Orders



In [19]:
import pandas as pd

orders = pd.read_csv('customer_orders.csv')
orders

,OrderID,CustomerID,Product,Quantity,Price
0,1,101,Laptop,2,800
1,2,102,Headphones,1,150
2,3,103,Smartphone,3,400
3,4,101,External Hard Drive,2,80
4,5,102,Backpack,1,40
...,...,...,...,...,...
95,96,103,Blender,1,60
96,97,104,Sport Shoes,2,40
97,98,105,Comforter Set,1,45
98,99,101,Wireless Earbuds,2,120


In [23]:
#1.Group the data by CustomerID and filter out customers who have made less than 20 orders.

active_customers = orders.groupby('CustomerID')['OrderID'].nunique()
filt = active_customers[active_customers >= 20].index

print(active_customers)

CustomerID
101    21
102    21
103    20
104    20
105    18
Name: OrderID, dtype: int64


In [25]:
#2.Identify customers who have ordered products with an average price per unit greater than $120.

customer_avg_price = orders.groupby('CustomerID')['Price'].mean()
high_value_customers = customer_avg_price[customer_avg_price > 120].index

print(high_value_customers)

Index([102, 104], dtype='int64', name='CustomerID')


In [26]:
#3.Find the total quantity and total price for each product ordered, and filter out products that have a total quantity less than 5 units

# Group by Product and sum Quantity & Price
product_stats = orders.groupby('Product').agg(
    Total_Quantity=('Quantity', 'sum'),
    Total_Price=('Price', 'sum')
).reset_index()

# Filter out products with Total_Quantity <5
filtered_products = product_stats[product_stats['Total_Quantity'] >= 5]

print(filtered_products)

             Product  Total_Quantity  Total_Price
5        Cargo Pants               6          120
15       Dress Shirt               5           50
19      Formal Shirt               6          105
30        Smartphone               5          800
32       Sport Shoes               5          120
35        Sunglasses               5           60
41  Wireless Earbuds               6          360


## Homework Assignment 3: Population Salary Analysis

In [ ]:
import sqlite3
import pandas as pd
import numpy as np

salary_bands = pd.read_excel('task/population salary analysis.xlsx', sheet_name='Salary Bands')

conn = sqlite3.connect('task/population.db')
query = "SELECT * FROM population"
population_df = pd.read_sql(query, conn)
conn.close()

def categorize_salary(salary):
    for _, band in salary_bands.iterrows():
        if band['Min'] <= salary < band['Max']:
            return band['Category']
    return 'Unknown'

population_df['Salary Category'] = population_df['Salary'].apply(categorize_salary)

def calculate_measures(df):
    measures = {
        'Percentage of Population': df.groupby('Salary Category').size() / len(df) * 100,
        'Average Salary': df.groupby('Salary Category')['Salary'].mean(),
        'Median Salary': df.groupby('Salary Category')['Salary'].median(),
        'Population Count': df.groupby('Salary Category').size()
    }
    return pd.concat(measures, axis=1)

overall_results = calculate_measures(population_df)

if 'State' in population_df.columns:
    state_results = population_df.groupby('State').apply(calculate_measures)
    state_results = state_results.unstack(level=1)
else:
    state_results = "No State information in the dataset"

print("Overall Results:")
print(overall_results)

print("\nResults by State:")
print(state_results)